# Figures for the 2025 publication


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from SALib.analyze import sobol
from SALib.sample import sobol as sobol_sample

from mxlpy import Model, Simulator, make_protocol, mc, plot

## Kirschbaum 1988 model

In [ ]:
def _nrh(a: float, b: float, c: float) -> float:
    return (b - np.sqrt(b**2 - 4 * a * c)) / (2 * a)


def _seq(par: float, alphags: float, thetags: float, s_min: float) -> float:
    return _nrh(thetags, alphags * par + 1 + s_min, alphags * par + s_min)


def _vvx(
    vxeq: float,
    vx: float,
    taui: float,
    taud: float,
) -> float:
    tau = taui if vx <= vxeq else taud
    return (vxeq - vx) / tau


def _vpi(s: float, pi: float, tau_k: float) -> float:
    return (s - pi) / tau_k


def _vw(pi: float, w: float, tau_h20: float) -> float:
    return (pi - w) / tau_h20


def _gs(w: float, gmax: float, gmin: float) -> float:
    return max(gmax * w, gmin)


def get_kirschbaum_1988() -> Model:
    """Get the Kirschbaum 1988 model.

    @article{kirschbaum1988observed,
      title={Observed and modelled stomatal responses to dynamic light environments in the shade plant Alocasia macrorrhiza},
      author={Kirschbaum, MUF and Gross, LJ and Pearcy, RW},
      journal={Plant, Cell & Environment},
      volume={11},
      number={2},
      pages={111--121},
      year={1988},
      publisher={Wiley Online Library}
    }

    """
    return (
        Model()
        .add_parameters(
            {
                "alphags": 0.0088,  # 640.0, # KirschBaum uses weird light unit
                "gmax": 71.5,
                "gmin": 13.6,
                "Smin": 0.192,
                "thetags": 0.866,
                "taui": 22.2,
                "taud": 420.0,
                "tauK": 780.0,  # taupi
                "tauH2O": 900.0,  # tauw
                "PAR": 100.0,
            }
        )
        .add_variables({"S": 0, "pi": 0, "w": 0})
        .add_derived(name="Seq", fn=_seq, args=["PAR", "alphags", "thetags", "Smin"])
        .add_derived(name="gs", fn=_gs, args=["w", "gmax", "gmin"])
        .add_reaction(
            name="Vs",
            fn=_vvx,
            args=["Seq", "S", "taui", "taud"],
            stoichiometry={"S": 1},
        )
        .add_reaction(
            name="Vpi", fn=_vpi, args=["S", "pi", "tauK"], stoichiometry={"pi": 1}
        )
        .add_reaction(
            name="Vw", fn=_vw, args=["pi", "w", "tauH2O"], stoichiometry={"w": 1}
        )
    )

## Steady state in darkness

used as initial condition for further simulation

In [ ]:
y0_dark_adapted = (
    Simulator(get_kirschbaum_1988().update_parameter("PAR", 0.000001))
    .simulate_to_steady_state()
    .get_result()
    .unwrap_or_err()
).get_new_y0()

## Sanity check: replicate original figure 6

### Figure 6

In [ ]:
protocol = make_protocol(
    [
        (10 * 60, {"PAR": 0.00}),
        (5 * 60, {"PAR": 100}),
        (100 * 60, {"PAR": 0.000}),
    ]
)

concs, fluxes = (
    Simulator(get_kirschbaum_1988().update_variables(y0_dark_adapted))
    .simulate_protocol(protocol=protocol, time_points_per_step=100)
    .get_result()
    .unwrap_or_err()
)

s_norm = concs["S"].max()

fig, left = plot.lines(concs / s_norm)
plot.shade_protocol(protocol["PAR"], ax=left, alpha=0.1)
left.set(
    xlabel="time / a.u.",
    ylabel="concentration / a.u.",
    xlim=(0, 90 * 60),
    ylim=(0, 1.2),
)
plot.show()

# Sensitivity analysis

- Input selection: all $\tau$

- Range: $X \sim \mathcal{N}\left(\mu,\, \left(\frac{p\% \cdot \mu}{300}\right)^2\right)$, where $p\%$ is the percentage around the mean in which 99.7 % of the data points live

- Sensitivity analysis method: Sobol sensitivity analysis

- Design of experiment: Sobol sequence

In [ ]:
def sigma3(p: float, m: float) -> float:
    return p * m / 300.0


p = 10.0  # percent
plst = ["taui", "taud", "tauK", "tauH2O"]
n_samples = 2**10

problem = {
    "names": ["taui", "taud", "tauK", "tauH2O"],
    "num_vars": 4,
    "bounds": [
        [20.0, sigma3(20.0, 20.0)],
        [450.0, sigma3(20.0, 450.0)],
        [780.0, sigma3(20.0, 780.0)],
        [900.0, sigma3(20.0, 900.0)],
    ],
    "dists": ["norm", "norm", "norm", "norm"],
}

salib_sample = sobol_sample.sample(problem, n_samples)
paramter_df = pd.DataFrame([dict(zip(plst, i, strict=True)) for i in salib_sample])

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for col, left in zip(paramter_df.columns, axes.flatten(), strict=True):
    paramter_df[col].plot.kde(ax=left)
    left.set_title(col)
    left.set_xlabel("Value")
plt.tight_layout()
plt.show()

In [ ]:
# Run Monte-Carlo Simulation
protocol3 = make_protocol(
    [
        (3 * 60, {"PAR": 0.00}),
        (5 * 60, {"PAR": 500}),
        (90 * 60, {"PAR": 0.00}),
        (5 * 60, {"PAR": 500}),
        (90 * 60, {"PAR": 0.00}),
    ]
)


tc = mc.protocol(
    get_kirschbaum_1988().update_variables(y0_dark_adapted),
    time_points_per_step=100,
    protocol=protocol3,
    mc_to_scan=paramter_df,
)

fig, left = plot.lines_mean_std_from_2d_idx(tc.variables.dropna(), names=["gs"])
left.set(xlabel="time / [s]", ylabel="gs")

In [ ]:
# Reformate data
clst = [tc.variables.loc[i]["gs"] for i in range(len(salib_sample))]

time_concs_df = pd.concat(clst, axis=1).T

s1_result = pd.DataFrame(
    np.zeros((len(time_concs_df.columns), len(plst))), columns=plst
)

st_result = pd.DataFrame(
    np.zeros((len(time_concs_df.columns), len(plst))), columns=plst
)

for index, i in enumerate(time_concs_df.columns):
    Y = time_concs_df[i]
    Si = sobol.analyze(problem, Y.values, print_to_console=False)
    s1_result.iloc[index, :] = Si["S1"]
    st_result.iloc[index, :] = Si["ST"]
    s1_result.index = time_concs_df.columns
    st_result.index = time_concs_df.columns

In [ ]:
fig, left = plot.one_axes(figsize=(8, 5))
right = left.twinx()

s1_result.rename(
    columns={
        "taui": r"$\tau_i$",
        "taud": r"$\tau_d$",
        "tauK": r"$\tau_{\Pi}$",
        "tauH2O": r"$\tau_W$",
    }
).plot(ax=left, linewidth=2, color=["grey", "purple", "orange", "red"])


plot.lines_mean_std_from_2d_idx(tc.variables.dropna(), names=["gs"], ax=right)
plot.shade_protocol(protocol3["PAR"], ax=left, alpha=0.1, add_legend=False)
left.set(
    xlabel="time / [s]",
    ylabel="First-order sensitivity",
    ylim=(0, 1.1),
    xlim=(0, 11600),
)
right.set(
    ylabel=r"gs [mmol $\cdot m^2 \cdot s^{-1}$]",
)
left.grid()
right.grid()
left.legend(bbox_to_anchor=(1.0, 0.75), loc="upper right")
right.legend(bbox_to_anchor=(1.0, 0.95), loc="center right")
plot.show()
fig.savefig("K88SA.png", dpi=300, bbox_inches="tight")